In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from nltk.corpus import stopwords
import itertools
import nltk
import re
tqdm.pandas()
import ast

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

nltk.download('stopwords')

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /cluster/home/yakram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
LANGUAGE = 'de'
DATA_DIR = 'german'

In [3]:
triplet_save_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/{LANGUAGE}/hard_neg_triplets.csv'
triplet_save_dir

'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/de/hard_neg_triplets.csv'

In [4]:
triplet_classification_data = pd.read_csv(triplet_save_dir, sep='\t', index_col=[0])

In [5]:
positive_classification_data = triplet_classification_data[
    ['?a', 'anchor', 'positive_term']
]

negative_classification_data = triplet_classification_data[
    ['?a', 'anchor', 'negative_term']
]

In [6]:
positive_classification_data['labels'] = 1
negative_classification_data['labels'] = 0

In [7]:
positive_classification_data.columns = ['?a', 'term1', 'term2', 'labels']
negative_classification_data.columns = ['?a', 'term1', 'term2', 'labels']

In [8]:
classification_data_unshuffled = pd.concat([positive_classification_data, negative_classification_data])

In [9]:
classification_data_unshuffled.shape

(14374, 4)

In [10]:
classification_data_unshuffled.drop_duplicates(inplace=True)
classification_data_unshuffled.shape

(14143, 4)

In [11]:
classification_data_unshuffled_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{LANGUAGE}/unshuffled_classification_data.csv'
classification_data_shuffled_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{LANGUAGE}/shuffled_classification_data.csv'

In [12]:
classification_data_unshuffled_dir, classification_data_shuffled_dir

('/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/de/unshuffled_classification_data.csv',
 '/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/de/shuffled_classification_data.csv')

In [13]:
classification_data_unshuffled.to_csv(classification_data_unshuffled_dir, sep='\t')

In [14]:
classification_data_unshuffled = classification_data_unshuffled.sample(frac=1)

In [15]:
first_half_unshuffled = classification_data_unshuffled[: int(len(classification_data_unshuffled)/2)]
second_half_unshuffled = classification_data_unshuffled[int(len(classification_data_unshuffled)/2) :]

In [16]:
first_half_unshuffled.shape, second_half_unshuffled.shape, classification_data_unshuffled.shape

((7071, 4), (7072, 4), (14143, 4))

In [17]:
second_half_unshuffled.head()

,?a,term1,term2,labels
3251,http://data.europa.eu/esco/skill/7b406d9a-3887...,Anlagen zum Erhitzen von Metallen bedienen,Verwenden von Anlagen zum Brennen der ausgegos...,1
4093,http://data.europa.eu/esco/skill/9b9dc550-351e...,Verfahren des Vertriebsmanagements überprüfen,Entwickeln und Überprüfen von Vertriebsverfahr...,1
1907,http://data.europa.eu/esco/skill/497b85db-c3ba...,Unterstützung beim Ankleiden leisten,Hilfestellung beim Auswählen von Kleidung für ...,1
3481,http://data.europa.eu/esco/skill/849c8e8f-0416...,Schleifmaschine bedienen,"Bedienen und Überwachen der Schleifmaschine, u...",1
6423,http://data.europa.eu/esco/skill/f780a6ad-b01e...,Krisenmanagement anwenden,Übernahme der Kontrolle über Pläne und Strateg...,1


In [18]:
second_half_shuffled = second_half_unshuffled[['?a', 'term2', 'term1', 'labels']]
second_half_shuffled.columns = ['?a', 'term1', 'term2', 'labels']

In [19]:
classification_data_shuffled = pd.concat([first_half_unshuffled, second_half_shuffled])
classification_data_shuffled.shape

(14143, 4)

In [20]:
classification_data_shuffled.to_csv(classification_data_shuffled_dir, sep='\t')